In [5]:
%pip install opencv-python
%pip install tensorflow


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import cv2
import numpy as np
import time
import tensorflow as tf

# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="model.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Set up the webcam stream.
cap = cv2.VideoCapture(0)

# Set up the text display parameters.
font = cv2.FONT_HERSHEY_SIMPLEX
org = (50, 50)
fontScale = 1
color = (0, 255, 0)
thickness = 2

# Initialize the frame counter and FPS calculation.
frame_count = 0
start_time = time.time()

# Set up the video recording parameters.
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = None
recording = False
recording_start_time = 0

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Preprocess the input image.
    resized = cv2.resize(frame, (256, 256), interpolation = cv2.INTER_AREA)
    normalized = resized.astype('float32') / 255.0
    input_data = np.expand_dims(normalized, axis=0)
    
    # Perform the inference.
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    
    # Get the predicted class and confidence.
    prediction = output_data[0][0]
    if prediction < 0.5:
        label = 'Fire'
        if frame_count == 0:
            recording = True
            recording_start_time = time.time()
    else:
        label = 'No fire'
        if recording:
            recording = False
            if out is not None:
                out.release()
                out = None
    
    # Display the prediction and frame rate on the screen.
    cv2.putText(frame, label, org, font, fontScale, color, thickness, cv2.LINE_AA)
    frame_count += 1
    if frame_count % 10 == 0:
        end_time = time.time()
        fps = int(frame_count / (end_time - start_time))
        print('FPS:', fps)
        accuracy = round(prediction * 100, 2)
        print('Accuracy:', accuracy, '%')
        frame_count = 0
        start_time = end_time
    
    # Save the frames if fire is detected.
    if recording:
        if out is None:
            out = cv2.VideoWriter('video.mp4', fourcc, fps, (frame.shape[1], frame.shape[0]))
        out.write(frame)
        elapsed_time = time.time() - recording_start_time
        if elapsed_time > 10 or out.get(cv2.CAP_PROP_POS_FRAMES) * out.get(cv2.CAP_PROP_FPS) * 0.000001 > 2:
            recording = False
            out.release()
            out = None
    
    # Display the resulting frame.
    cv2.imshow('Fire Detection',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

FPS: 11
Accuracy: 81.07 %
FPS: 28
Accuracy: 82.25 %
FPS: 28
Accuracy: 83.01 %
FPS: 29
Accuracy: 83.33 %
FPS: 24
Accuracy: 81.63 %
FPS: 31
Accuracy: 83.28 %
FPS: 30
Accuracy: 83.78 %
FPS: 26
Accuracy: 83.56 %
FPS: 23
Accuracy: 82.19 %
FPS: 19
Accuracy: 83.81 %
FPS: 25
Accuracy: 82.68 %
FPS: 26
Accuracy: 86.5 %
FPS: 30
Accuracy: 83.32 %
FPS: 36
Accuracy: 85.75 %
FPS: 29
Accuracy: 88.83 %
FPS: 22
Accuracy: 90.81 %
FPS: 28
Accuracy: 89.58 %
FPS: 29
Accuracy: 88.69 %
FPS: 30
Accuracy: 90.21 %
FPS: 29
Accuracy: 90.55 %
FPS: 29
Accuracy: 88.51 %
FPS: 30
Accuracy: 86.12 %
FPS: 27
Accuracy: 89.64 %
FPS: 32
Accuracy: 92.3 %
FPS: 28
Accuracy: 94.93 %
FPS: 30
Accuracy: 96.59 %
FPS: 35
Accuracy: 98.02 %
FPS: 27
Accuracy: 97.83 %
FPS: 28
Accuracy: 98.16 %
FPS: 32
Accuracy: 97.47 %
FPS: 29
Accuracy: 97.35 %
FPS: 28
Accuracy: 97.48 %
FPS: 32
Accuracy: 97.55 %
FPS: 29
Accuracy: 97.7 %
FPS: 29
Accuracy: 97.62 %
FPS: 32
Accuracy: 97.53 %
FPS: 28
Accuracy: 97.13 %
FPS: 28
Accuracy: 97.43 %
FPS: 27
Accurac